In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
TRAIN_PATH = 'digital_reputation_challenge_train/train/'
TEST_PATH = 'digital_reputation_challenge_test/test/'

Загрузим данные

In [3]:
d_train, d_test = {}, {}
all_urls = []
all_urls_num = 0

In [4]:
with open(TRAIN_PATH + 'Y.csv', 'r') as f:
    for line in f:
        if 'id' in line:
            continue
        lst = line[:-1].split(',')
        d_train[lst[0]] = [np.array([float(s) for s in lst[1:]])]

In [5]:
with open(TRAIN_PATH + 'X2.csv', 'r') as g:
    for line in g:
        if 'id' in line:
            continue
        idx, site = line[:-1].split(',')
        d_train[idx].append(site)
        all_urls.append(site)
        all_urls_num += 1

In [6]:
with open(TEST_PATH + 'X2.csv', 'r') as f:
    for line in f:
        if 'id' in line:
            continue
        idx, site = line[:-1].split(',')
        if idx not in d_test.keys():
            d_test[idx] = [site]
        else:
            d_test[idx].append(site)

In [7]:
import collections
all_urls = collections.Counter(all_urls)

In [ ]:
d = {'id': [], '1': [], '2': [], '3': [], '4': [], '5': []}
for test_id in d_test.keys():
    num = 0.
    res = np.array([0., 0., 0., 0., 0.])
    for train_id in d_train.keys():
        common = set(d_train[train_id][1:]) & set(d_test[test_id])
        m = 0
        for url in common: 
            m += ((math.log(all_urls_num) - math.log(all_urls[url])) - 6)
        res += d_train[train_id][0] * m
        num += m
    d['id'].append(test_id)
    if num < 1:
        for i in '12345':
            d[i].append(0.3)
    else:
        res /= num
        for i in range(1, 6):
            d[str(i)].append(res[i - 1])

In [ ]:
first_solution_result = pd.DataFrame(data=d)

In [ ]:
first_solution_result

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import (KFold, StratifiedKFold, cross_val_score,
                                     cross_validate, train_test_split)
from xgboost import XGBClassifier

In [ ]:
X1 = pd.read_csv(TRAIN_PATH + 'X1.csv')
X1_test = pd.read_csv(TEST_PATH + 'X1.csv')
Y = pd.read_csv(TRAIN_PATH + 'Y.csv')

In [ ]:
X1['2'] = [int(val) for val in X1['2']]
X1['3'] = [int(val) for val in X1['3']]
X1_test['2'] = [int(val) for val in X1_test['2']]
X1_test['3'] = [int(val) for val in X1_test['3']]

In [ ]:
d_mx = {'min_child_samples': 100, 'learning_rate': 0.010, 'max_depth': 3, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 195}
d_mx_2 = {'min_child_samples': 100, 'learning_rate': 0.019, 'max_depth': 1, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 200}
d_mx_3 = {'min_child_samples': 100, 'learning_rate': 0.012, 'max_depth': 2, 'max_delta_step': 2, 'num_leaves': 12, 'min_child_weight': 2, 'n_estimators': 195}
d_mx_4 = {'learning_rate': 0.018, 'max_delta_step': 2, 'max_depth': 1, 'min_child_samples': 150, 'min_child_weight': 7, 'n_estimators': 315, 'num_leaves': 20}

In [ ]:
cls = []
cls.append(XGBClassifier(n_jobs=8, random_state=0, objective='rank:pairwise', **d_mx))
cls.append(XGBClassifier(n_jobs=8, random_state=0, **d_mx_2))
cls.append(XGBClassifier(n_jobs=8, random_state=0, objective='rank:pairwise', **d_mx_3))
cls.append(XGBClassifier(n_jobs=8, random_state=0, **d_mx_4))

In [ ]:
probas = []
for i in range(4):
    cls[i].fit(X1.values, Y[str(i+1)].values)
    proba = cls[i].predict_proba(X1_test.values)[:, 1]
    probas += [proba]

In [ ]:
second_solution_result = pd.DataFrame(probas).T

In [ ]:
second_solution_result

In [ ]:
coef_list = [0.3, 0.7, 0.3, 0.7]

for i in range(4):
    
    first_lst = first_solution_result[str(i + 1)]
    first_np = np.array(first_lst)
    first_lst = [(first_np > val).sum() for val in first_lst]

    second_lst = second_solution_result[i]
    second_np = np.array(second_lst)
    second_lst = [(second_np > val).sum() for val in second_lst]
    
    final_ranks = (1 - (np.array(first_lst) * coef_list[i] + np.array(second_lst) * (1 - coef_list[i])) / 4060).tolist()
    first_solution_result[str(i + 1)] = final_ranks

In [ ]:
first_solution_result.to_csv('final.csv', index=False)